In [1]:
import numpy as np
from itertools import product
import math
import random
import copy

According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

#### ISING MODEL

In [2]:
class IsingModel:
    def __init__(self, data, lr=0.0005):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(data)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # First calculate the correlation
        self.data_correlation = self.calculate_corr(self.data_activations_matrix)
        # Then, the mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
        
        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        np.random.seed(3)
        self.J = np.random.normal(loc=0.0, scale=.01, size=self.data_correlation.shape)
        lind = np.tril_indices_from(self.J)
        self.J[lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
        
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            model_correlation = np.zeros(self.J.shape)
            model_mean = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                
                # Calculate the expectation of the correlations
                for i in range(0, self.num_spins):
                    for j in range(i+1, self.num_spins):
                        model_correlation[i][j] += sigma[i]*sigma[j]*prob_dict[sigma]

                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    model_mean[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            # Take the step
            oldJ = copy.deepcopy(self.J)
            self.J = self.J + stepJ

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Calculate the variation of J for the termination condition
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
            
            epoch += 1

        print(self.H, 'H')
        print(self.J, 'J')

    # Calculate the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            for j in range(i+1, self.num_spins):
                energy += sigma[i]*sigma[j]*self.J[i][j]

        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    def get_delta_energy(self, i):
        #delta_energy = 2*(self.H[spin_to_flip]*self.sigma[spin_to_flip])
        
        delta_energy= 2 * (self.H[i]*self.sigma[i] \
                           + self.sigma[i] * np.dot(self.J[i, :], self.sigma) \
                           + self.sigma[i] * np.dot(self.J[:, i], self.sigma))
        
        return delta_energy
    
    def metropolis_step(self):
        # Randomly select a spin (now it can only select 0)
        spin_to_flip = np.random.randint(self.num_spins)

        #delta_energy = 2*(self.H[spin_to_flip]*self.sigma[spin_to_flip])
        delta_energy = self.get_delta_energy(spin_to_flip)

        p = math.exp(-delta_energy)
        r = random.random()

        #print(delta_energy, r, p)
        if delta_energy <= 0 or r<=p:
            self.sigma[spin_to_flip] = -self.sigma[spin_to_flip]

    def metropolis_simulation(self, max_steps, burn_in_len=0):
        # Generate a random initial spin configuration
        self.sigma = random.choices(self.spin_values, k=self.num_spins)        
        
        # Create a dictionary to count the frequencies
        samples_dict = {}
        for i in range(0, max_steps):
            # Take a metropolis step
            self.metropolis_step()
            
            # Save the state if the burn-in phase has passed
            if i >= burn_in_len:
                
                if not tuple(copy.deepcopy(self.sigma)) in samples_dict: # If the state is new, create an entry
                    samples_dict[tuple(copy.deepcopy(self.sigma))] = 1
                else: # If the state already appeared, add one
                    samples_dict[tuple(copy.deepcopy(self.sigma))] += 1

        # Normalize the probability
        sum_states = sum(samples_dict.values())
        normalized_dict = {k: v / sum_states for k, v in samples_dict.items()}
                                                 
        return normalized_dict

    def get_statistics(self, states_dict):
        # Returns the magnetization and correlation from a dictionary of activations {sigma, prob}
        magnetization =  np.zeros(self.num_spins)
        corr_matrix = np.zeros((self.num_spins,self.num_spins))
        # Calculate the expectation
        for sigma, prob in states_dict.items():
            magnetization += np.dot(sigma,prob)
            for i in range(0, self.num_spins):
                for j in range(i+1, self.num_spins):
                    corr_matrix[i][j] += sigma[i]*sigma[j]*prob
        return magnetization, corr_matrix
    
    # Calculates the correlation of a list of activations
    def calculate_corr(self, activations_matrix):
        num_samples_data = activations_matrix.shape[0]
        
        corr_matrix = np.zeros((self.num_spins, self.num_spins))
        for sigma in activations_matrix:
            # Calculate \sigma_i * \sigma_j
            for i in range(0, self.num_spins):
                for j in range(i+1, self.num_spins):
                    corr_matrix[i][j] += sigma[i]*sigma[j]
        corr_matrix = corr_matrix / float(num_samples_data)
        
        return corr_matrix
    
    def train_metropolis(self, max_epochs = 300, simulation_len = 100, burn_in_len=0):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0000000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:

            samples_dict = self.metropolis_simulation(simulation_len, burn_in_len)
        
            # Calculate the data expectations        
            model_mean, model_correlation = self.get_statistics(samples_dict)
            
            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            oldJ = copy.deepcopy(self.J)
            # Take the step
            self.J = self.J + stepJ
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            # Calculate the variation of J and H for the termination condition
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%50 == 0 or epoch == max_epochs-1:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
                
            epoch +=1

        
        print(self.data_mean, 'Spins average')
        print(self.H, 'H')
        
    

In [3]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]
s_ising = IsingModel(activations_list, lr=0.01)

In [4]:
s_ising.train_metropolis( max_epochs = 5000, simulation_len = 1000, burn_in_len=50)

Epoch 50 TotalParamVariation 0.01910526
[[ 0.         -0.15368421  0.08421053 -0.11157895]
 [ 0.          0.         -0.04210526 -0.08631579]
 [ 0.          0.          0.          0.02105263]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.01894737 -0.06526316  0.14736842  0.09894737] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 100 TotalParamVariation 0.008
[[ 0.         -0.32842105  0.22947368 -0.13263158]
 [ 0.          0.         -0.24       -0.09684211]
 [ 0.          0.          0.          0.01894737]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.23157895 -0.35157895  0.37894737  0.06736842] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 150 TotalParamVariation 0.00675789
[[ 0.

Epoch 950 TotalParamVariation 0.00463158
[[ 0.         -0.49473684  0.19578947 -0.18736842]
 [ 0.          0.         -0.20842105 -0.2       ]
 [ 0.          0.          0.          0.03157895]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.19578947 -0.26736842  0.44421053  0.09052632] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1000 TotalParamVariation 0.00469474
[[ 0.         -0.58736842  0.21263158 -0.22105263]
 [ 0.          0.         -0.26736842 -0.11578947]
 [ 0.          0.          0.         -0.03157895]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.21473684 -0.23578947  0.49684211 -0.08      ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1050 TotalParamVariation 0.006589

Epoch 1800 TotalParamVariation 0.00281053
[[ 0.         -0.51578947  0.33263158 -0.23368421]
 [ 0.          0.         -0.29473684 -0.21684211]
 [ 0.          0.          0.          0.00421053]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.26736842 -0.25473684  0.54736842  0.01473684] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1850 TotalParamVariation 0.00604211
[[ 0.         -0.43157895  0.10947368 -0.26526316]
 [ 0.          0.         -0.14315789 -0.21894737]
 [ 0.          0.          0.          0.01894737]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.15157895 -0.17263158  0.52421053  0.02315789] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1900 TotalParamVariation 0.00518

Epoch 2650 TotalParamVariation 0.00594737
[[ 0.         -0.44210526  0.31789474 -0.28421053]
 [ 0.          0.         -0.20631579 -0.26526316]
 [ 0.          0.          0.         -0.09894737]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.29052632 -0.18315789  0.56842105 -0.10105263] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 2700 TotalParamVariation 0.00848421
[[ 0.         -0.48421053  0.09894737 -0.14736842]
 [ 0.          0.         -0.13052632 -0.33052632]
 [ 0.          0.          0.         -0.01684211]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.12210526 -0.11157895  0.55157895 -0.04421053] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 2750 TotalParamVariation 0.00618

Epoch 3500 TotalParamVariation 0.00448421
[[ 0.         -0.47157895  0.20842105 -0.23368421]
 [ 0.          0.         -0.17263158 -0.28210526]
 [ 0.          0.          0.         -0.01473684]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.27157895 -0.31157895  0.39368421  0.04842105] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 3550 TotalParamVariation 0.00283158
[[ 0.         -0.46105263  0.26315789 -0.20631579]
 [ 0.          0.         -0.22526316 -0.29052632]
 [ 0.          0.          0.          0.00842105]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.24       -0.17684211  0.49263158 -0.02315789] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 3600 TotalParamVariation 0.0034


Epoch 4400 TotalParamVariation 0.01284211
[[ 0.         -0.57052632  0.30736842 -0.11368421]
 [ 0.          0.         -0.37894737 -0.29894737]
 [ 0.          0.          0.          0.14526316]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.48210526 -0.54526316  0.58526316  0.08421053] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 4450 TotalParamVariation 0.00601053
[[ 0.         -0.56        0.32842105 -0.22315789]
 [ 0.          0.         -0.27578947 -0.19578947]
 [ 0.          0.          0.         -0.04842105]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.4        -0.34315789  0.52842105 -0.03578947] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 4500 TotalParamVariation 0.00665

In [ ]:
s_ising.train(1000)